# 프로젝트: Movielens 영화 추천 

### 1) 데이터 준비와 전처리

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'like'}, inplace=True)

In [4]:
ratings['like']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: like, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
#movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = "ISO-8859-1")
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# 검색하기 쉽게 하기위해 타이틀 문자열을 소문자로 변경
movies['title'] = movies['title'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


#### 여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.



### 2) 분석해 봅시다.

### ratings에 있는 유니크한 영화 개수

In [7]:
ratings['movie_id'].nunique()

3628

### rating에 있는 유니크한 사용자 수

In [8]:
ratings['user_id'].nunique()

6039

In [9]:
#movie_id 를 영화title 변경 하여 가장 인기 있는 영화에 타이틀이 나오게 했다.
ratings['movie_id'] = ratings['movie_id'].replace(movies['title'])

### 가장 인기 있는 영화 30개(인기순)

In [10]:
# 인기 많은 영화 30개  평점 순으로 해보았다.
movie_count_rating = ratings.groupby('movie_id')['like'].mean()
movie_count_rating.sort_values(ascending=False).head(30)

movie_id
passion in the desert (1998)                                                         5.000000
perils of pauline, the (1947)                                                        5.000000
cup, the (phörpa) (1999)                                                             5.000000
old lady who walked in the sea, the (vieille qui marchait dans la mer, la) (1991)    5.000000
mchale's navy (1997)                                                                 5.000000
eraserhead (1977)                                                                    5.000000
guess who's coming to dinner (1967)                                                  5.000000
grease 2 (1982)                                                                      5.000000
kurt & courtney (1998)                                                               5.000000
in the company of men (1997)                                                         5.000000
bluebeard (1944)                                   

### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다

In [11]:
data = ratings[['user_id','movie_id','like']]

In [12]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['toystroy' , 'popeye' ,'aerican pop' ,'jumanji' ,'kansas city']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zoo']*5, 'movie_id': my_favorite, 'like':[5]*5})

if not data.isin({'user_id':['zoo']})['user_id'].any():  # user_id에 'zoo'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,like
1000203,6040,"leopard son, the (1996)",3
1000205,6040,bird of prey (1996),5
1000206,6040,naked in new york (1994),5
1000207,6040,palookaville (1996),4
1000208,6040,"associate, the (1996)",4
0,zoo,toystroy,5
1,zoo,popeye,5
2,zoo,aerican pop,5
3,zoo,jumanji,5
4,zoo,kansas city,5


In [13]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['movie_id'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [14]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zoo'])     
print(movie_to_idx['toystroy'])

6039
3628


In [15]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing OK!!')
    data['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

data

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,like
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,3628,5
1,6039,3629,5
2,6039,3630,5
3,6039,3631,5


### 4) CSR matrix를 직접 만들어 봅시다.

In [16]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()

csr_data = csr_matrix((data.like, (data.user_id, data.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [17]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [18]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=500, regularization=0.01, iterations=20, dtype=np.float32)

In [19]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [20]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [21]:
zoo, toystroy = user_to_idx['zoo'], movie_to_idx['toystroy']
zoo_vector, toystroy_vector = als_model.user_factors[zoo], als_model.item_factors[toystroy]


In [22]:
zoo_vector

array([-1.00055775e-02,  3.30425464e-02,  4.84537631e-02,  3.54891159e-02,
       -7.79318511e-02,  3.45491432e-02,  2.01229863e-02,  9.04862061e-02,
        9.71890390e-02, -8.97653326e-02,  1.34921027e-02, -3.75794582e-02,
        7.65581280e-02,  7.42955953e-02,  9.47581884e-03, -3.80961969e-02,
       -8.13019462e-03, -5.16437218e-02,  1.38025172e-03,  5.05908839e-02,
        4.94531505e-02,  5.45191690e-02, -5.05679287e-02, -7.62317330e-02,
       -1.03169814e-01, -1.29668964e-02,  3.92061695e-02, -5.68940490e-03,
       -5.55710867e-02,  8.13654512e-02, -2.09841356e-02,  4.89810482e-02,
       -1.16881952e-01,  1.38062105e-01, -2.39697602e-02, -5.99178486e-02,
        3.75517718e-02,  1.89830884e-02, -1.27234552e-02,  1.36025503e-01,
        2.05538645e-02,  4.94926376e-03, -1.71212345e-01,  1.51520111e-02,
       -2.40954235e-02,  1.13207430e-01,  6.86866418e-02,  9.97573957e-02,
        4.20662425e-02, -4.99061383e-02,  9.18352082e-02,  1.28624991e-01,
        1.49312261e-02,  

In [23]:
toystroy_vector

array([0.00509867, 0.00534324, 0.00568539, 0.00437975, 0.00404919,
       0.00467329, 0.00440965, 0.00589443, 0.00649453, 0.00317433,
       0.00498681, 0.00361062, 0.00525026, 0.00552714, 0.00554699,
       0.00498408, 0.00456329, 0.00401604, 0.00446073, 0.00563225,
       0.00559914, 0.00551298, 0.00469253, 0.00401748, 0.00285344,
       0.00470557, 0.0054926 , 0.00520207, 0.00446531, 0.00597684,
       0.00462907, 0.00556042, 0.00298261, 0.00705054, 0.00513531,
       0.00455692, 0.00496604, 0.00550711, 0.0051003 , 0.00691549,
       0.00425135, 0.0047346 , 0.00338179, 0.00426134, 0.00439399,
       0.00680801, 0.006507  , 0.00625531, 0.00618271, 0.00485265,
       0.00640265, 0.00613465, 0.00439619, 0.00500374, 0.00543562,
       0.00502907, 0.0043051 , 0.00493025, 0.00264361, 0.0022515 ,
       0.00431479, 0.00637502, 0.00527489, 0.00287782, 0.00479724,
       0.00370139, 0.00510424, 0.00599343, 0.00689529, 0.00369477,
       0.00545213, 0.00508746, 0.00442505, 0.00404705, 0.00425

In [24]:
np.dot(zoo_vector, toystroy_vector)

0.030502481

In [25]:
toystroy = movie_to_idx['toystroy']
movie_vector = als_model.item_factors[toystroy]
np.dot(zoo_vector, toystroy_vector)

0.030502481

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [26]:
favorite_movie = 'toystroy'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3628, 1.0000001),
 (3631, 0.9999929),
 (3629, 0.9999928),
 (3632, 0.9999924),
 (3630, 0.9999915),
 (3615, 0.99928516),
 (3531, 0.9988539),
 (3549, 0.9988487),
 (3035, 0.998755),
 (3589, 0.9985748),
 (3570, 0.99854547),
 (3591, 0.99850225),
 (3088, 0.99837846),
 (3053, 0.9983739),
 (3534, 0.9982943)]

In [27]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['toystroy',
 'jumanji',
 'popeye',
 'kansas city',
 'aerican pop',
 'she-devil (1989)',
 'sweet nothing (1995)',
 'beautiful (2000)',
 'kicked in the head (1997)',
 'breaking away (1979)',
 'perfect candidate, a (1996)',
 'dear jesse (1997)',
 'federal hill (1994)',
 'bananas (1971)',
 'spiral staircase, the (1946)']

In [28]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [29]:
get_similar_movie('toystroy')

['toystroy',
 'jumanji',
 'popeye',
 'kansas city',
 'aerican pop',
 'she-devil (1989)',
 'sweet nothing (1995)',
 'beautiful (2000)',
 'kicked in the head (1997)',
 'breaking away (1979)']

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [30]:
user = user_to_idx['zoo']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(1959, 0.03358883),
 (1135, 0.032080594),
 (3540, 0.031101359),
 (3549, 0.03097365),
 (570, 0.030823864),
 (3242, 0.030808963),
 (682, 0.030781887),
 (3296, 0.030674584),
 (3591, 0.030449357),
 (3569, 0.030373149),
 (2459, 0.03027369),
 (3611, 0.030256577),
 (3332, 0.0301555),
 (3353, 0.030139249),
 (3364, 0.029978737),
 (3616, 0.02995363),
 (3168, 0.029894285),
 (3589, 0.029866945),
 (2303, 0.029845105),
 (2160, 0.029809294)]

In [31]:
[idx_to_movie[i[0]] for i in movie_recommended]

['creature (1999)',
 'carriers are waiting, the (les convoyeurs attendent) (1999)',
 'lethal weapon 4 (1998)',
 'beautiful (2000)',
 'attack of the killer tomatoes! (1980)',
 'echte kerle (1996)',
 'surviving the game (1994)',
 'great day in harlem, a (1994)',
 'dear jesse (1997)',
 'benji (1974)',
 'puppet master ii (1990)',
 'beauty and the beast (1991)',
 'titanic (1953)',
 'frogs for snakes (1998)',
 'madeline (1998)',
 "guess who's coming to dinner (1967)",
 'pagemaster, the (1994)',
 'breaking away (1979)',
 'prince valiant (1997)',
 'jefferson in paris (1995)']

### 9) 선호도 및 유사도 기여도 확인하기

In [32]:
madeline = movie_to_idx['madeline (1998)']
explain = als_model.explain(user, csr_data, itemid=madeline)

In [33]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('popeye', 0.005983627868543219),
 ('toystroy', 0.005971718205774279),
 ('aerican pop', 0.005936797529632529),
 ('jumanji', 0.005884568480362697),
 ('kansas city', 0.005645492898351744)]

# 프로젝트를 마치며... 

### 데이터 전처리 부분 아직 많이 미숙한것같다 개념들은 알아가는데 항상 사용하려구 하면 헷갈리고 어렵다.... 활용하고 응용 하기까지 더 많이 검색해보고 사용해 봐야 할것같다. 
### 데이터 전처리 이후는 크게 어려운 부분이 없었다. 이번 노드는 잘 읽으면서 따라가면서 하기 충분하였다. 